# Displaying a DataGrid

This example doesn't use the appinator, it only uses the underlying DataGrid widget.

Features shown:
- Display a table
- Set column metadata to control formatting
- Add a click handler for clicking on Cells

A few notes:
- Columns are, by default, Title Cased... so: sepal_length is displayed as Sepal Length
- Click events print to the Output widget at the top (callbacks need an Output context)

In [ ]:
import logging

import pandas as pd
from ipywidgets import Output

from nbappinator import FORMAT_DECIMAL, FORMAT_PERCENT, apply_format, create_grid, get_column_defs

logger = logging.getLogger(__name__)

# Output widget to capture click events (print in callbacks doesn't show without this)
out = Output()
display(out)

In [ ]:
grid_data = {"species": ["something"] * 100, "sepal_length": range(100), "some_perc": range(100)}

# To use a more complete dataset:
# data = pd.read_csv('https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv')

create_grid(pd.DataFrame(grid_data), showindex=True)

In [ ]:
data = pd.DataFrame(grid_data)

In [ ]:
# Same data, but with column metadata definitions

cols = get_column_defs(data)
cols[0]["pinned"] = "left"  # species
apply_format(cols[1], FORMAT_DECIMAL, precision=3)  # sepal_length
cols[1]["headerName"] = "Length of Sepal"
apply_format(cols[2], FORMAT_PERCENT, precision=3)  # some_perc (divide by 100)
cols[2]["headerName"] = "Some Percentage"

# Note: For perc_div equivalent, manually divide the data
data_pct = data.copy()
data_pct["some_perc"] = data_pct["some_perc"] / 100

grid = create_grid(
    data_pct,
    column_defs=cols,
)
grid

In [ ]:
# With click events

cols = get_column_defs(data)
cols[0]["pinned"] = "left"  # species
apply_format(cols[1], FORMAT_DECIMAL, precision=3)  # sepal_length
cols[1]["headerName"] = "Length of Sepal"
cols[2]["hide"] = True  # some_perc - hidden

click_events = []


def cell_clicked(evt):
    click_events.append(evt)
    with out:
        print(f"cell clicked {evt}")


grid3 = create_grid(
    data,
    column_defs=cols,
    action=cell_clicked,
)

# Test that handler works
grid3.process_message({"event_type": "cellClicked", "col_clicked": "species", "value_clicked": "test"})
assert len(click_events) == 1, "Click handler should have been called"
print("Click handler test passed")

grid3

In [ ]:
# Use standard column definitions

cols = get_column_defs(data)
apply_format(cols[1], FORMAT_DECIMAL, precision=3)  # sepal_length
cols[1]["headerName"] = "Length of Sepal"

grid_b = create_grid(
    data,
    column_defs=cols,
)
grid_b

In [ ]:
# With multi-select

multi_click_events = []


def cell_clicked_multi(evt):
    multi_click_events.append(evt)
    with out:
        print(f"multi-select cell clicked {evt}")


multiple_grid = create_grid(
    data,
    select_mode="multiple",
    action=cell_clicked_multi,
)

# Test that handler works
multiple_grid.process_message({"event_type": "cellClicked", "col_clicked": "species", "value_clicked": "test"})
assert len(multi_click_events) == 1, "Multi-select click handler should have been called"
print("Multi-select click handler test passed")

display(multiple_grid)

In [ ]:
# Grid without action handler (display only)

display_only_grid = create_grid(
    data,
    select_mode="single",
)
print("Display-only grid created (no click handler)")
display_only_grid